In [1]:
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [ ]:
import torch
print(torch.__version__, torch.cuda.is_available())
!gcc --version

In [3]:
from ultralytics import YOLO
from IPython.display import display, Image
from IPython import display
import glob
import cv2
import matplotlib.pyplot as plt
display.clear_output()

Train the model

In [ ]:
!yolo task=detect mode=train model=yolov8m.pt pretrained=True data=Annotations/data.yaml epochs=25 lr0=0.01 imgsz=640 plots=True project=runs

Validate the model

In [6]:
!yolo task=detect mode=val model=/home/habtamu/Substrate/YOLOv8/runs/train/weights/best.pt data=Annotations/data.yaml project=runs

Ultralytics YOLOv8.1.26 🚀 Python-3.10.9 torch-1.13.1+cu117 CUDA:0 (NVIDIA A40, 45413MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning /home/habtamu/Substrate/YOLOv8/Annotations/val/labels.cache... 101
val: WARNING ⚠️ /home/habtamu/Substrate/YOLOv8/Annotations/val/images/012946_I0000055_b.png: ignoring corrupt image/label: negative label values [   -0.83124    -0.58129]
val: WARNING ⚠️ /home/habtamu/Substrate/YOLOv8/Annotations/val/images/012996_I0000604_b.png: ignoring corrupt image/label: negative label values [   -0.70685    -0.95099]
val: WARNING ⚠️ /home/habtamu/Substrate/YOLOv8/Annotations/val/images/013206_I0000982_b.png: ignoring corrupt image/label: negative label values [   -0.99967    -0.99967]
val: WARNING ⚠️ /home/habtamu/Substrate/YOLOv8/Annotations/val/images/013765_I0000189_b.png: ignoring corrupt image/label: negative label values [   -0.38078    -0.68132]
val: WARNING ⚠️ /home/habtamu/Substrate/YOLOv8/Annotations/val/im

Perform the inference

In [ ]:
!yolo task=detect mode=predict model=/home/habtamu/Substrate/YOLOv8/runs/train/weights/best.pt conf=0.75 source=Annotations/test/images save=True project=runs

Get bbox coordinates

In [ ]:
model = YOLO('runs/train/weights/best.pt')
validation_results = model.predict(source='Annotations/val/images', conf=0.67, project='runs')

In [59]:
print(len(validation_results))
print(validation_results[0].boxes.xyxy)
print(validation_results[0].boxes.conf)
print(validation_results[0].boxes.cls)
tensor_result = validation_results[0].boxes.xyxy
numpy_array_result = tensor_result.cpu().numpy()
print(numpy_array_result)
flattened_array = numpy_array_result.flatten()
print(flattened_array)

101
tensor([[ 453.3691,  471.5865, 2640.4343, 2493.6985]], device='cuda:0')
tensor([0.9740], device='cuda:0')
tensor([0.], device='cuda:0')
[[     453.37      471.59      2640.4      2493.7]]
[     453.37      471.59      2640.4      2493.7]


Crop images based on the predicted bbox

In [4]:
model = YOLO('runs/train/weights/best.pt')

In [5]:
def crop_images(original_images_folder, cropped_images_folder, results):
    i = 0
    # Get a list of filenames in the folder and sort them
    filenames = sorted(os.listdir(original_images_folder))
    for filename in filenames:
        if filename.endswith('.png'):
            #print("i=", i)
            input_file_path = os.path.join(original_images_folder, filename)
            output_file_path = os.path.join(cropped_images_folder,filename)
            img = cv2.imread(input_file_path)
            bbox_tensor_result = results[i].boxes.xyxy
            if bbox_tensor_result.shape[0] >= 2:
                bbox_numpy_result = bbox_tensor_result[0].cpu().numpy().flatten()
            else:
                bbox_numpy_result = bbox_tensor_result.cpu().numpy().flatten()
            if bbox_numpy_result.shape[0]!=0:
                x_min, y_min, x_max, y_max = bbox_numpy_result
            else:
                # Get image dimensions
                height, width, _ = img.shape
                x_min, y_min, x_max, y_max = 0, 0, width, height
            
            #img_copy = img.copy()
            # display the image with its roi bbox
            #cv2.rectangle(img_copy, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (255, 0, 0), 8)
            #plt.imshow(img_copy)
            #plt.show()
        
            # Crop and save the image
            img = img[int(y_min):int(y_max), int(x_min):int(x_max)]
            cv2.imwrite(output_file_path,img)
            i = i + 1

In [61]:
# crop validation images
original_images_folder = "Annotations/val/images"
cropped_images_folder = "Cropped_val_images"
crop_images(original_images_folder, cropped_images_folder,validation_results)

In [ ]:
# crop OK_bluecare_feb24 images
ok_bluecare_feb24_results = model.predict(source='/home/habtamu/data/substrate/OK_bluecare_feb24/prova', conf=0.67, project='runs')
original_images_folder = "/home/habtamu/data/substrate/OK_bluecare_feb24/prova"
cropped_images_folder = "/home/habtamu/data/substrate/OK_bluecare_feb24_cropped"
crop_images(original_images_folder, cropped_images_folder,ok_bluecare_feb24_results)

In [ ]:
# crop OK_cullera_feb24 images
ok_cullera_feb24_results = model.predict(source='/home/habtamu/data/substrate/OK_cullera_feb24/prova', conf=0.67, project='runs')
original_images_folder = "/home/habtamu/data/substrate/OK_cullera_feb24/prova"
cropped_images_folder = "/home/habtamu/data/substrate/OK_cullera_feb24_cropped"
crop_images(original_images_folder, cropped_images_folder,ok_cullera_feb24_results)

In [ ]:
# crop KO_bluecare images
ko_bluecare_results = model.predict(source='/home/habtamu/data/substrate/KO_bluecare/prova', conf=0.67, project='runs')
original_images_folder = "/home/habtamu/data/substrate/KO_bluecare/prova"
cropped_images_folder = "/home/habtamu/data/substrate/KO_bluecare_cropped"
crop_images(original_images_folder, cropped_images_folder,ko_bluecare_results)

In [ ]:
# crop KO_cullera images
ko_cullera_results = model.predict(source='/home/habtamu/data/substrate/KO_cullera/prova', conf=0.67, project='runs')
original_images_folder = "/home/habtamu/data/substrate/KO_cullera/prova"
cropped_images_folder = "/home/habtamu/data/substrate/KO_cullera_cropped"
crop_images(original_images_folder, cropped_images_folder, ko_cullera_results)